<a href="https://colab.research.google.com/github/vappanna/BerAIML/blob/main/prompt_III.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

17 campaigns

### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [3]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
import time

In [29]:
df = pd.read_csv('/content/bank-additional-full.csv', sep = ';')

In [30]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```



### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

**Business Objective**
The business goal is to find a model that can explain success of a contact, i.e. if the client subscribes the deposit. Such model can increase campaign efficiency by identifying the main characteristics that affect success, helping in a better management of the available resources (e.g. human effort, phone calls, time) and selection of a high quality and affordable set of potential buying customers.

### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features, prepare the features and target column for modeling with appropriate encoding and transformations.

In [41]:
# 1. Define the Education hierarchy (Ordinal)
edu_mapping = {
    'illiterate': 0, 'basic.4y': 1, 'basic.6y': 2, 'basic.9y': 3,
    'high.school': 4, 'professional.course': 5, 'university.degree': 6, 'unknown': 2 # Using median for unknown
}
df['education_encoded'] = df['education'].map(edu_mapping)

# 2. One-Hot Encode Nominal variables (Job, Marital, Housing, Loan, Default)
# We use drop_first=True to avoid the "Dummy Variable Trap" (multicollinearity)
nominal_cols = ['job', 'marital', 'default', 'housing', 'loan','duration']
df_encoded = pd.get_dummies(df[nominal_cols], drop_first=True)

# 3. Encode the Target (y)
df['target'] = df['y'].map({'yes': 1, 'no': 0})

# 4. Combine with numeric 'age'
X = pd.concat([df[['age', 'education_encoded']], df_encoded], axis=1)
# Dropping duration and the original target to isolate features
#X = df.drop(columns=['duration', 'y'])
y = df['target']
df_encoded

,duration,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,...,job_unknown,marital_married,marital_single,marital_unknown,default_unknown,default_yes,housing_unknown,housing_yes,loan_unknown,loan_yes
0,261,False,False,True,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
1,149,False,False,False,False,False,False,True,False,False,...,False,True,False,False,True,False,False,False,False,False
2,226,False,False,False,False,False,False,True,False,False,...,False,True,False,False,False,False,False,True,False,False
3,151,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
4,307,False,False,False,False,False,False,True,False,False,...,False,True,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,334,False,False,False,False,True,False,False,False,False,...,False,True,False,False,False,False,False,True,False,False
41184,383,True,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
41185,189,False,False,False,False,True,False,False,False,False,...,False,True,False,False,False,False,False,True,False,False
41186,442,False,False,False,False,False,False,False,False,True,...,False,True,False,False,False,False,False,False,False,False


### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

In [42]:
# 1. Isolate only the features you want (Bank Client Data 1-7)
# This automatically "drops" duration and everything else (8-20)
bank_features = ['age', 'job', 'marital', 'education', 'default', 'housing', 'loan']
X = df[bank_features].copy()

# 2. Set your target
y = df['y'].map({'yes': 1, 'no': 0})

# 3. Transform X (Encoding education, dummies for jobs, etc.)
X_encoded = pd.get_dummies(X, drop_first=True)

# 4. NOW split the clean data
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, stratify=y)
print(f"Training rows: {X_train.shape[0]}")
print(f"Testing rows: {X_test.shape[0]}")

Training rows: 32950
Testing rows: 8238


In [43]:
print("Target distribution in Training Set:")
print(y_train.value_counts(normalize=True))

print("\nTarget distribution in Testing Set:")
print(y_test.value_counts(normalize=True))

Target distribution in Training Set:
y
0    0.887344
1    0.112656
Name: proportion, dtype: float64

Target distribution in Testing Set:
y
0    0.887351
1    0.112649
Name: proportion, dtype: float64


### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

In [47]:
# Calculate the percentage of the majority class
baseline_accuracy = y_train.value_counts(normalize=True).max()
print(f"Baseline Accuracy to beat: {baseline_accuracy:.2%}")

Baseline Accuracy to beat: 88.73%


In [51]:
# We must scale our numeric features

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [1]:
def evaluate_model(model, X_train, y_train, X_test, y_test):
    # Measure Train Time
    start = time.time()
    model.fit(X_train, y_train)
    end = time.time()
    train_time = end - start

    # Calculate Accuracies
    train_acc = accuracy_score(y_train, model.predict(X_train))
    test_acc = accuracy_score(y_test, model.predict(X_test))
    return train_time, train_acc, test_acc






### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.  

In [76]:
lr = LogisticRegression(random_state=42)
#lr.fit(X_train_scaled, y_train)
results_list = []
t, tr_acc, te_acc = evaluate_model(lr, X_train_scaled, y_train, X_test_scaled, y_test)
results_list.append({
        "Model": "LogisiticRegression",
        "Train Time (s)": t,
        "Train Accuracy": tr_acc,
        "Test Accuracy": te_acc
    })


[{'Model': 'LogisiticRegression',
  'Train Time (s)': 0.0704185962677002,
  'Train Accuracy': 0.8873444613050075,
  'Test Accuracy': 0.8873512988589464}]

### Problem 9: Score the Model

What is the accuracy of your model?

In [63]:
# y_test are the true labels, y_pred are the model's guesses
accuracy = accuracy_score(y_test, y_pred_log)

print(f"The LogisticRegression model accuracy is: {accuracy:.2%}")

The LogisticRegression model accuracy is: 88.74%


### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|     |    |.     |.     |

KNN Classifier

In [77]:
# Initialize and fit
knn = KNeighborsClassifier(n_neighbors=5)
#knn.fit(X_train_scaled, y_train)

# Predict
#y_pred_knn = knn.predict(X_test_scaled)
t, tr_acc, te_acc = evaluate_model(knn, X_train_scaled, y_train, X_test_scaled, y_test)
results_list.append({
        "Model": "KNN",
        "Train Time (s)": t,
        "Train Accuracy": tr_acc,
        "Test Accuracy": te_acc
          })
results_list

[{'Model': 'LogisiticRegression',
  'Train Time (s)': 0.0704185962677002,
  'Train Accuracy': 0.8873444613050075,
  'Test Accuracy': 0.8873512988589464},
 {'Model': 'KNN',
  'Train Time (s)': 0.007919788360595703,
  'Train Accuracy': 0.8892867981790592,
  'Test Accuracy': 0.875455207574654}]

In [ ]:
accuracy = accuracy_score(y_test, y_pred_knn)

print(f"The KNN Classifier model accuracy is: {accuracy:.2%}")

In [ ]:
# Create the DataFrame
df_results = pd.DataFrame(results_list)

# Calculate the "Overfit Gap"
df_results['Accuracy Gap'] = df_results['Train Accuracy'] - df_results['Test Accuracy']

print(df_results.sort_values(by="Test Accuracy", ascending=False))

Decision Tree Classifier

In [78]:
# 1. Initialize the model
# We limit depth so the tree doesn't grow infinitely
dtree = DecisionTreeClassifier(max_depth=5, random_state=42)

t, tr_acc, te_acc = evaluate_model(dtree, X_train_scaled, y_train, X_test_scaled, y_test)
results_list.append({
        "Model": "Decsion Tree",
        "Train Time (s)": t,
        "Train Accuracy": tr_acc,
        "Test Accuracy": te_acc
          })
results_list

[{'Model': 'LogisiticRegression',
  'Train Time (s)': 0.0704185962677002,
  'Train Accuracy': 0.8873444613050075,
  'Test Accuracy': 0.8873512988589464},
 {'Model': 'KNN',
  'Train Time (s)': 0.007919788360595703,
  'Train Accuracy': 0.8892867981790592,
  'Test Accuracy': 0.875455207574654},
 {'Model': 'Decsion Tree',
  'Train Time (s)': 0.044137001037597656,
  'Train Accuracy': 0.8876176024279211,
  'Test Accuracy': 0.887229910172372}]

SVM

In [79]:
# 1. Initialize the model
# 'rbf' (Radial Basis Function) is the standard non-linear kernel
svm_model = SVC(kernel='rbf', probability=True, random_state=42)

t, tr_acc, te_acc = evaluate_model(svm_model, X_train_scaled, y_train, X_test_scaled, y_test)
results_list.append({
        "Model": "SVM",
        "Train Time (s)": t,
        "Train Accuracy": tr_acc,
        "Test Accuracy": te_acc
          })
results_list

[{'Model': 'LogisiticRegression',
  'Train Time (s)': 0.0704185962677002,
  'Train Accuracy': 0.8873444613050075,
  'Test Accuracy': 0.8873512988589464},
 {'Model': 'KNN',
  'Train Time (s)': 0.007919788360595703,
  'Train Accuracy': 0.8892867981790592,
  'Test Accuracy': 0.875455207574654},
 {'Model': 'Decsion Tree',
  'Train Time (s)': 0.044137001037597656,
  'Train Accuracy': 0.8876176024279211,
  'Test Accuracy': 0.887229910172372},
 {'Model': 'SVM',
  'Train Time (s)': 273.7870817184448,
  'Train Accuracy': 0.8877086494688923,
  'Test Accuracy': 0.8875940762320952}]

### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.


- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.  
- Adjust your performance metric

In [97]:
# Create the DataFrame
df_results = pd.DataFrame(results_list)

# Calculate the "Overfit Gap"
df_results['Accuracy Gap'] = df_results['Train Accuracy'] - df_results['Test Accuracy']

print(df_results.sort_values(by="Test Accuracy", ascending=False))

                 Model  Train Time (s)  Train Accuracy  Test Accuracy  \
3                  SVM      273.787082        0.887709       0.887594   
0  LogisiticRegression        0.070419        0.887344       0.887351   
2         Decsion Tree        0.044137        0.887618       0.887230   
1                  KNN        0.007920        0.889287       0.875455   

   Accuracy Gap  
3      0.000115  
0     -0.000007  
2      0.000388  
1      0.013832  


##### Questions